# Compare Paypal Vs CM

###Specifications

####  PayPal:

Outcome		
Compare Paypal Vs CM 		
		
Input		
1. CM segregated file		
2. Paypal Raw file		
		
CM preprocessing		
1. Remove records that contains 'refund' in 'Charge ID'		
		
Paypal  Preprocessing		
"1. Filter non-revenue records: Remove the records with the following 'Types': 'Hold on Available Balance','Reversal of General Account Hold',
          'Hold on Balance for Dispute Investigation','Cancellation of Hold for Dispute Resolution',
          'General Payment','General Withdrawal','PreApproved Payment Bill User Payment',
          'Dispute Fee','Chargeback','Account Hold for Open Authorization','Payment Reversal','Chargeback Reversal',
          'Fee Reversal','Mass Pay Payment','Mass Pay Reversal','Payment Refund'"		
2. Remove 'General Currency Records'		
3.  Filter negative values under 'Subscription payment'		
4. For few records, 'Transaction Id' is mapped with part of strings under 'Charge Id' to get the CM value		
5. Compare 'Gross' with 'Line Item Value'		
		
Note: Refund recors are ignored as of now		

### Script


#### Imports,prepartions and functions

In [1]:
import pandas as pd
import os
!pip install XlsxWriter
import xlsxwriter


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 2.7 MB/s eta 0:00:00


###### Preparations
---


In [2]:
#Mount GDrive
from google.colab import drive
drive.mount('/content/gdrive')

#Set file paths
path = "/content/gdrive/My Drive/Data Science 2022/CM Audit/"

filepp = path + "Paypal Data/2023/Paypal_Raw_Data_2023_02.xlsx"
filecm  =path + "Monthly Output/2023.02/CM_2023.02.xlsx"
outpath = path + "Monthly Output/2023.02/Comparison Results/"
outfile = outpath + "PayPalVsCM_202302.xlsx"
#Create Results folder if missing
if not os.path.isdir(outpath):
  os.mkdir(outpath)


Mounted at /content/gdrive


#### Compare PP Vs CM 


In [3]:
lstType = ['Hold on Available Balance','Reversal of General Account Hold',
          'Hold on Balance for Dispute Investigation','Cancellation of Hold for Dispute Resolution',
          'General Payment','General Withdrawal','PreApproved Payment Bill User Payment',
          'Dispute Fee','Chargeback','Account Hold for Open Authorization','Payment Reversal','Chargeback Reversal',
          'Fee Reversal','Mass Pay Payment','Mass Pay Reversal','Payment Refund']


#For csv format
if filepp[-3:] == "csv":  
  df = pd.read_csv(filepp , na_filter=False, index_col=False)
else:
  df = pd.read_excel(filepp , na_filter=False, index_col=False)

#Change the data type of gross to numeric
df = df.astype({"Gross": str})
df['Gross'] = df['Gross'].str.replace(',','')
df = df.astype({"Gross": float})


############ Segregate Non-Revenue transactions based on 'Type'
#Move the below 'Types' to seperate sheet - Non revenue
#Hold on Available Balance,Reversal of General Account Hold,Hold on Balance for Dispute Investigation,Cancellation of Hold for Dispute Resolution,General Payment,General Withdrawal,
#PreApproved Payment Bill User Payment,Dispute Fee,Chargeback,Account Hold for Open Authorization,Payment Reversal,Chargeback Reversal,Fee Reversal,Mass Pay Payment,Mass Pay Reversal
dfmove = df[df['Type'].isin(lstType)] #dataset to be moved to 'others' sheet
dfclean = df[~df['Type'].isin(lstType)] #clean dataset

'''
############# Filter 'General currency conversion' with 'EUR'
#dfclean1 = dfclean[~((dfclean['Type'] == "General Currency Conversion") & (dfclean['Currency'] != "USD"))]
#dfeuro = dfclean[((dfclean['Type'] == "General Currency Conversion") & (dfclean['Currency'] != "USD"))]
dfclean1 = dfclean[~(dfclean['Currency'] != "USD")]
dfeuro = dfclean[(dfclean['Currency'] != "USD")]
dfmove1= dfmove.append(dfeuro)
'''

############# Filter all 'General currency conversion' records
dfclean1 = dfclean[~(dfclean['Type'] == "General Currency Conversion")]
dfeuro = dfclean[(dfclean['Type'] == "General Currency Conversion")]
#dfclean1 = dfclean[~(dfclean['Currency'] != "USD")]
#dfeuro = dfclean[(dfclean['Currency'] != "USD")]
dfmove1= dfmove.append(dfeuro)

############# Filter negative values under 'Subscription payment'
dfclean2 = dfclean1[~((dfclean1['Type'] == "Subscription Payment") & (dfclean1['Gross'] < 0))]
dfmove2 = dfmove1.append(dfclean1[((dfclean1['Type'] == "Subscription Payment") & (dfclean1['Gross'] < 0))])

############# **** Get 'Line Item Value' from CM file ** #####################
dfpp = dfclean2.copy()
dfpp.rename(columns={"Reference Txn ID": 'Customer External ID'}, inplace = True)
#Load CM file
dfcm0 = pd.read_excel(filecm,sheet_name="Paypal" , na_filter=False, index_col=False)

#@@@@@@@@@@@@@@ filter 'refund' records from CM @@@@@@@@@@@@@@@@
dfcm = dfcm0[~dfcm0['Charge ID'].str.contains('refund')]

dfmon = pd.merge(dfpp, dfcm, on="Customer External ID", how='left')

#Keep only paypal data along with plan
cmcols = dfcm.columns.tolist()
cmcols.remove('Plan Type')
cmcols.remove('Line Item Value')
#cmcols.remove('Line Item Value Account Currency')

cmcols.remove('Customer External ID')
mcols = dfmon.columns.tolist()
newcols = [x for x  in mcols if x not in cmcols  ]
dfnewpp = dfmon[newcols]

#Rename paypal column back to 'Reference Txn ID' 
dfnewpp.rename(columns={'Customer External ID': 'Reference Txn ID'}, inplace = True)

#Delete duplicate rows
dfnewpp = dfnewpp.drop_duplicates()
'''
############## Fill out plan type for 'general currency conversion' #################
#1. Pick 'General currency conversion' records in cleaned dataset
#2. Map "Reference Txn ID" with the 'Transaction ID' of 'non revenue' records and find 
#   corresponding 'Ref Txn ID' . This id will start with 'I-'
#3. Map this new "Reference Txn ID" with CM data to get the plan
#####################################################################################
dfcurr = dfnewpp[dfnewpp['Type'] == "General Currency Conversion"]    #Pick 'General currency conversion' records in cleaned dataset
indall=[]
curr_dic = dfcurr.to_dict('records')    #convert to dictionary for iteration efficiency

for row in curr_dic:
  item = row['Reference Txn ID']            
  dftemp = dfmove2[dfmove2['Transaction ID'] == item]   #Map "Reference Txn ID" with the 'Transaction ID' of 'non revenue' records
  
  inds = dfmove2[dfmove2['Transaction ID'] == item].index   #Drop the corresponding records in non revenue records
  
  indall.append(inds[0])
  

  id = dftemp['Reference Txn ID'].tolist()[0]           #find corresponding 'Ref Txn ID' . This id will start with 'I-'
  plan = dfcm[dfcm['Customer External ID'] == id]['Plan Type']
  liv = dfcm[dfcm['Customer External ID'] == id]['Line Item Value']
  #livc = dfcm[dfcm['Customer External ID'] == id]['Line Item Value Account Currency'] 
  
  if not plan.empty:                                    #Subsciption refund will have empty values
    row['Plan Type'] = plan.values[0]
    row['Line Item Value'] = liv.values[0]
    #row['Line Item Value Account Currency'] = livc.values[0]

dfcurr = pd.DataFrame.from_dict(curr_dic)

dfmove2.drop (indall,inplace=True)        #Drop all curr records in non revenue dataset

#Append new currency conversion records 
dfnewpp = pd.concat([dfnewpp,dfcurr])
'''
##############################################################################################################################################
#few records that are without plan whose 'Transaction Id' is mapping with part of strings under 'Charge Id', Invoice Id'  and 'Line Item Name' of CM data. 
#Map those and fill out the plan type
##############################################################################################################################################
#Get records that are without plan type
#Map 'ref txn id' of these records with the CM's ChatgeId column to find if the string is contained in it. If so, pick the plan
#dfempty = dfnewpp[dfnewpp['Plan Type'].isnull()]
#dfnewpp = dfnewpp[~dfnewpp['Plan Type'].isnull()]

dict_emp = dfnewpp.to_dict('records')     
#cnt=0 
for row in dict_emp:
  x = row['Transaction ID']
  dfchargeid=pd.DataFrame()
  dfchargeid = dfcm[dfcm['Charge ID'].str.contains(str(x))]   #If charge id contains ref txn id then pick the plan type from that row
  if not dfchargeid.empty:
    #cnt += 1
    dfchargeid.reset_index(drop=True, inplace=True)
    row['Plan Type'] = dfchargeid.iloc[0]['Plan Type']
    row['Line Item Value'] = dfchargeid.iloc[0]['Line Item Value']
    #row['Line Item Value Account Currency'] = dfchargeid.iloc[0]['Line Item Value Account Currency']
#dffilled = pd.DataFrame.from_dict(dict_emp)
#print(cnt)
dfnewpp = pd.DataFrame.from_dict(dict_emp)

#dfnewpp = pd.concat([dfnewpp,dffilled])

dfnewpp.drop_duplicates(inplace=True)
#Rearrange column order. Move 'Gross' near 'Line Item Value'
new_cols = [col for col in dfnewpp.columns if col != 'Gross'] + ['Gross']
dfnewpp = dfnewpp[new_cols]

new_cols1 = [col for col in dfnewpp.columns if col != 'Line Item Value'] + ['Line Item Value']
dfnewpp = dfnewpp[new_cols1]

#Add difference column
diff = dfnewpp['Gross'] - dfnewpp['Line Item Value']
dfnewpp['Difference'] = diff

dfnewpp.rename(columns={'Line Item Value':'From CM data'}, inplace=True)
#Sort by plan type
dfnewpp.sort_values('Plan Type',ascending=False,inplace=True)

writer = pd.ExcelWriter(outfile, engine='xlsxwriter')
dfnewpp.to_excel(writer,sheet_name="PayPal Vs CM", index=False)
dfmove2.to_excel(writer,sheet_name="Non Revenue records", index=False)

writer.save()
writer.close()    
    

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/usr/local/lib/python3.8/dist-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
